In [138]:
from constants import SHARED_RANDOM_STATE
import numpy as np
from db_helper_functions import (
    get_stock_news_with_finbert_scores_from_db,
    get_stock_news_with_finbert_tone_scores_from_db,
    get_stock_news_with_finbert_whole_article_scores_from_db,
    get_stock_news_with_finbert_tone_whole_article_scores_from_db,
    get_stock_news_with_bertopic_sentiment_scores_from_db,
)

from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from supervised_learning_data_preprocessing_functions import (
    gen_df_for_supervised_learning,
)
from grid_model_search_functions import (
    iterative_grid_cv_model_testing,
)

ticker = "AAPL"

In [116]:
finbert_summary_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_scores_from_db,
)
finbert_tone_summary_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_tone_scores_from_db,
)
finbert_whole_article_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_whole_article_scores_from_db,
)
finbert_tone_article_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_tone_whole_article_scores_from_db,
)
bert_topic_article_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_bertopic_sentiment_scores_from_db,
)

/Users/raulmartinez/Desktop/MADS/Milestone_2/w24-milestone2-team18-leevw-steveso-raulmart/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/raulmartinez/Desktop/MADS/Milestone_2/w24-milestone2-team18-leevw-steveso-raulmart/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/raulmartinez/Desktop/MADS/Milestone_2/w24-milestone2-team18-leevw-steveso-raulmart/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

In [117]:
save_results = False
dataframes_to_test = [
    (bert_topic_article_sentiment_df, "bert_topic_article_sentiment_df"),
]
cv_train_size = [126]
cv_test_size = [31]
lag_time = [5]
tss_splits = 5
features_to_use = [
    "open",
    "prev_high",
    "prev_low",
    "prev_close",
    "positive",
    "negative",
    "neutral",
]


feature_to_predict = ["closed_higher"]
scoring_method = ["accuracy"]

param_grid = {
    "data_frame": dataframes_to_test,
    "cv_train_size": cv_train_size,
    "cv_test_size": cv_test_size,
    "lag_time": lag_time,
    "feature_to_predict": feature_to_predict,
    "scoring_method": scoring_method,
}
data_settings_grid_list = list(ParameterGrid(param_grid))

In [118]:
dum_clf_results = iterative_grid_cv_model_testing(
    model=DummyClassifier(),
    model_parameters={},
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
    tss_splits=tss_splits,
)

In [119]:
dum_clf_results = dum_clf_results.sort_values("mean_test_score", ascending=False)
dum_clf_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
0,0.001028,0.000158,0.00055,0.000036,{},0.548387,0.322581,0.516129,0.419355,0.419355,0.445161,0.080062,1,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [120]:
log_reg_clf_results = iterative_grid_cv_model_testing(
    model=LogisticRegression(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
    tss_splits=tss_splits,
)

In [121]:
log_reg_clf_results = log_reg_clf_results.sort_values(
    "mean_test_score", ascending=False
)
log_reg_clf_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
0,0.002107,0.000091,0.000703,0.000083,1337,{'model__random_state': 1337},0.548387,0.548387,0.548387,0.354839,0.516129,0.503226,0.075238,1,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [122]:
gb_clf_results = iterative_grid_cv_model_testing(
    model=GradientBoostingClassifier(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "learning_rate": [0.1, 0.2, 0.3, 0.4],
        "n_estimators": [5, 10, 20, 100],
        "max_depth": [None, 2, 5, 10, 15, 20],
        "max_leaf_nodes": [None, 2, 5, 10, 15, 20],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
    tss_splits=tss_splits,
)

In [123]:
gb_clf_results = gb_clf_results.sort_values(
    ["mean_test_score", "std_test_score"], ascending=[False, True]
)
gb_clf_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__learning_rate,param_model__max_depth,param_model__max_leaf_nodes,param_model__n_estimators,param_model__random_state,params,...,split4_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
482,0.022526,0.006917,0.001394,0.000745,0.4,5,None,20,1337,"{'model__learning_rate': 0.4, 'model__max_dept...",...,0.677419,0.625806,0.126754,1,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
207,0.065581,0.009132,0.000909,0.000038,0.2,5,10,100,1337,"{'model__learning_rate': 0.2, 'model__max_dept...",...,0.645161,0.619355,0.089861,2,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
71,0.067361,0.002947,0.000929,0.000014,0.1,5,20,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,0.612903,0.612903,0.064516,3,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
15,0.073872,0.008298,0.000918,0.000017,0.1,None,10,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,0.580645,0.606452,0.119660,4,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
87,0.074238,0.013119,0.000956,0.000080,0.1,10,10,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,0.580645,0.606452,0.119660,4,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [124]:
rf_clf_results = iterative_grid_cv_model_testing(
    model=RandomForestClassifier(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "n_estimators": [4, 5, 10, 15, 20, 50, 100],
        "max_depth": [None, 2, 5, 10, 15, 20],
        "max_leaf_nodes": [None, 2, 5, 10, 15, 20],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
    tss_splits=tss_splits,
)

In [125]:
rf_clf_results = rf_clf_results.sort_values("mean_test_score", ascending=False)
rf_clf_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__max_leaf_nodes,param_model__n_estimators,param_model__random_state,params,split0_test_score,...,split4_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
124,0.031272,0.011894,0.001430,0.000048,5,20,50,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.483871,...,0.580645,0.574194,0.055499,1,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
125,0.048675,0.004118,0.002028,0.000027,5,20,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.516129,...,0.580645,0.574194,0.031606,1,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
90,0.052866,0.010281,0.002043,0.000042,5,None,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.548387,...,0.580645,0.567742,0.032897,3,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
118,0.051057,0.006576,0.003260,0.002434,5,15,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.516129,...,0.580645,0.567742,0.048279,3,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
111,0.053973,0.006718,0.002054,0.000021,5,10,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.548387,...,0.580645,0.554839,0.024140,5,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [126]:
knn_clf_results = iterative_grid_cv_model_testing(
    model=KNeighborsClassifier(),
    model_parameters={
        "n_neighbors": [2, 3, 4, 5, 15, 20],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
    tss_splits=tss_splits,
)

In [127]:
knn_clf_results = knn_clf_results.sort_values("mean_test_score", ascending=False)
knn_clf_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
5,0.001385,0.000056,0.001511,0.000043,20,{'model__n_neighbors': 20},0.548387,0.451613,0.580645,0.548387,0.612903,0.548387,0.053978,1,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
1,0.001403,0.000094,0.001489,0.000039,3,{'model__n_neighbors': 3},0.548387,0.451613,0.516129,0.548387,0.645161,0.541935,0.062551,2,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
4,0.001450,0.000025,0.001568,0.000010,15,{'model__n_neighbors': 15},0.548387,0.548387,0.483871,0.483871,0.612903,0.535484,0.048279,3,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001385,0.000088,0.001483,0.000049,2,{'model__n_neighbors': 2},0.548387,0.354839,0.483871,0.516129,0.580645,0.496774,0.077955,4,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
3,0.001334,0.000075,0.001492,0.000049,5,{'model__n_neighbors': 5},0.548387,0.548387,0.290323,0.516129,0.516129,0.483871,0.097844,5,bert_topic_article_sentiment_df,126,31,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [128]:
if save_results:
    log_reg_clf_results.to_csv(
        "./model_results/logistic_regression_classifier_results.csv"
    )
    gb_clf_results.to_csv("./model_results/gb_classifier_results.csv")
    rf_clf_results.to_csv("./model_results/rf_classifier_results.csv")
    knn_clf_results.to_csv("./model_results/knn_classifier_results.csv")

### Total Models Tested


In [129]:
(
    len(dum_clf_results)
    + len(knn_clf_results)
    + len(gb_clf_results)
    + len(rf_clf_results)
    + len(log_reg_clf_results)
)

836

### Model Results


In [130]:
for df, name in [
    (dum_clf_results, "dum_clf_results"),
    (log_reg_clf_results, "log_reg_clf_results"),
    (knn_clf_results, "knn_clf_results"),
    (rf_clf_results, "rf_clf_results"),
    (gb_clf_results, "gb_clf_results"),
]:
    print(name)
    print(round(df.iloc[0].mean_test_score, 5))
    print(round(df.iloc[0].std_test_score, 5))
    print("#####")

dum_clf_results
0.44516
0.08006
#####
log_reg_clf_results
0.50323
0.07524
#####
knn_clf_results
0.54839
0.05398
#####
rf_clf_results
0.57419
0.0555
#####
gb_clf_results
0.62581
0.12675
#####


### Sentiment Performance Comparsion


In [131]:
dataframes_to_test = [
    (finbert_summary_sentiment_df, "finbert_summary_sentiment_df"),
    (finbert_tone_summary_sentiment_df, "finbert_tone_summary_sentiment_df"),
    (finbert_whole_article_sentiment_df, "finbert_whole_article_sentiment_df"),
    (finbert_tone_article_sentiment_df, "finbert_tone_article_sentiment_df"),
    (bert_topic_article_sentiment_df, "bert_topic_article_sentiment_df"),
]
param_grid = {
    "data_frame": dataframes_to_test,
    "cv_train_size": cv_train_size,
    "cv_test_size": cv_test_size,
    "lag_time": lag_time,
    "feature_to_predict": feature_to_predict,
    "scoring_method": scoring_method,
}
data_settings_grid_list = list(ParameterGrid(param_grid))

In [139]:
gb_clf_results = iterative_grid_cv_model_testing(
    model=GradientBoostingClassifier(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "learning_rate": [0.1, 0.2, 0.3, 0.4],
        "n_estimators": [5, 10, 20, 100],
        "max_depth": [None, 2, 5, 10, 15, 20],
        "max_leaf_nodes": [None, 2, 5, 10, 15, 20],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
    tss_splits=tss_splits,
)

In [140]:
for df, df_name in dataframes_to_test:
    score = (
        gb_clf_results[gb_clf_results.data_frame == df_name]
        .sort_values(["mean_test_score", "std_test_score"], ascending=[False, True])
        .iloc[0]
        .mean_test_score
    )
    std = (
        gb_clf_results[gb_clf_results.data_frame == df_name]
        .sort_values(["mean_test_score", "std_test_score"], ascending=[False, True])
        .iloc[0]
        .std_test_score
    )
    print(df_name, np.round(score, 5), np.round(std, 5))

finbert_summary_sentiment_df 0.58065 0.08893
finbert_tone_summary_sentiment_df 0.55484 0.06255
finbert_whole_article_sentiment_df 0.56129 0.06321
finbert_tone_article_sentiment_df 0.56774 0.07796
bert_topic_article_sentiment_df 0.62581 0.12675
